In [1]:
import tensorflow as tf

In [2]:
@tf.function
def to_prob_dist_all(v):
    v2 = tf.sqrt(tf.square(v)+1e-9)
    # v2 = tf.sqrt(tf.square(v))
    m = tf.expand_dims(tf.reduce_sum(v2, axis=-1),-1)
    n = tf.math.divide_no_nan(v2, m)
    return n

In [3]:
values = tf.constant([2,4,3], dtype=tf.float32)
choice = tf.Variable([1,0,0], dtype=tf.float32)

target = tf.constant([3], dtype=tf.float32)

opt = tf.keras.optimizers.Adam(3e-4)

steps = 1000

for i in range(steps):
    with tf.GradientTape() as tape:
        # s_choice = tf.nn.softmax(choice)
        s_choice = to_prob_dist_all(choice)
        quantized = values[tf.argmax(s_choice)]
        quantized = tf.tile(tf.expand_dims(quantized, 0),[3])
        out = s_choice + tf.stop_gradient(quantized - s_choice)
        out = tf.reduce_mean(out)

        loss = tf.nn.l2_loss(out - target)

    variables = [choice]
    grads = tape.gradient(loss, variables)

    opt.apply_gradients(zip(grads, variables))
    choice.assign(to_prob_dist_all(choice))

    if i % (steps // 10) == 0:
        tf.print(loss, out, grads)

0.5 2 [[5.96046448e-08 0 0]]
0.5 2 [[0 0 0]]
0.5 2 [[2.98023224e-08 2.96442728e-08 2.96442728e-08]]
0.5 2 [[5.96046412e-08 5.95177205e-08 5.95177205e-08]]
0.5 2 [[0 0 0]]
0.5 2 [[-2.98023224e-08 -2.97713303e-08 -2.97713303e-08]]
0.5 2 [[2.98023224e-08 2.97866176e-08 2.97866176e-08]]
0.5 2 [[-5.96046448e-08 -5.95884124e-08 -5.95884124e-08]]
0.5 2 [[0 0 0]]
0.5 2 [[2.98023224e-08 2.97949398e-08 2.97949398e-08]]
